## 7.19 结构与类

如今，编程教科书推荐使用面向对象编程，使软件开发更加清晰和模块化。
对象是结构和类的实例。
面向对象的编程风格对程序性能，既有正面又有负面的影响。积极影响是：

- 如果一组变量是相同结构或类的成员，则它们一起使用，也存储在一起。这使数据缓存更有效。
- 类成员变量不需要作为参数传递给类成员函数，避免了参数传输的开销。

面向对象编程的负面影响是：

- 非静态成员函数有一个'`this`'指针，它作为隐式参数传递给函数。所有非静态成员函数都会产生'`this`'参数传输的开销。
- '`this`'指针占用一个寄存器。寄存器是32位系统中的稀缺资源。
- 虚拟成员函数效率较低（请参阅第55页）。

**关于面向对象编程的正面影响，还是负面影响占主导地位，没有一般性的定论。**
至少，可以说使用类和成员函数，代价并不昂贵。
你可以使用面向对象的编程风格：
- 如果它对程序的逻辑结构和清晰度有好处
- 只要在程序的最关键部分避免过多的函数调用，

单纯结构的使用（没有成员函数）对性能没有负面影响。

## 7.20 类数据成员（实例变量）

无论在什么时间创建类或结构的实例，其数据成员都按照它们被声明的顺序连续存储。
将数据组织到类或结构中，并不会导致性能损失。
访问类或结构对象的数据成员并不会比访问简单变量话费更多的时间。

大多数编译器会将数据成员与地址对齐，以便优化访问，如下表所示。

Type | size, bytes  | alignment, bytes
 ---  | ---  | ---  |
bool  |  1   |  1
char, signed or unsigned  | 1  | 1
short int, signed or unsigned  | 2  | 2
int, signed or unsigned  | 4  | 4
64-bit integer, signed or unsigned  | 8  | 8
pointer or reference, 32-bit mode  | 4  | 4
pointer or reference, 64-bit mode  | 8  | 8
float  | 4  | 4
double  | 8 |  8
long double  | 8, 10, 12 or 16 |  8 or 16

Table 7.2. 数据成员的对齐

结构或类中如果混有不同大小的数据成员，数据对齐可能会导致未使用字节的空洞。例如：
```cpp
// Example 7.39a
struct S1 {
   short int a; // 2个字节. 第一个在偏移位置0, 第二个在位置1
   // 6个未使用的字节
   double b; // 8个字节. 第一个在偏移位置8, 最后一个在位置15
   int d; // 4个字节. 第一个在偏移位置16, 最后一个在位置19
   // 4个未使用字节
};
S1 ArrayOfStructures[100];
```
此处数据对齐导致结构缩小了8个字节，数组缩小了800个字节。

通过重新安排数据成员的编码顺序，通常可以使结构和类对象更小。
如果类有至少一个虚拟函数，则在第一个数据成员之前，或最后一个成员之后，有一个指向虚拟表的指针。
该指针在32位系统中为4个字节，在64位系统中为8个字节。
如果你对结构或其每个成员的大小有疑问，可以使用`sizeof`运算符进行测试。
`sizeof`运算符返回的值包括对象末尾的任何未使用的字节的大小。

- **如果数据成员相对于结构或类的开头的偏移量小于128，则生成的访问数据成员的代码更为紧凑，因为偏移量可以表示为8位有符号数。**
- 如果相对于结构或类的开头的偏移量大于等于128字节，则偏移量必须表示为32位数（指令集在介于8位和32位偏移之间，没有任何区别）。 

例如：
```cpp
// Example 7.40
class S2 {
public:
   int a[100]; // 400字节. 首字节偏移量0, 尾字节偏移量399
   int b; // 4字节. 首字节偏移量400, 尾字节偏移量403
   int ReadB() {return b;}
};
```

这里`b`的偏移量为400。通过指针或成员函数（如`ReadB`）访问`b`的任何代码都需要将偏移量编码为32位数。
如果交换`a`和`b`，则可以使用编码为8位有符号数的偏移量访问这两个变量，或者根本不用偏移。
这使代码更紧凑，从而更有效地使用代码缓存。
因此，对于大型数组和其他大型对象，建议在结构或类声明中排在最后。并且最常用的数据成员放在前面。
如果不能把所有数据成员包含前128个字节内，则将最常用的成员放在前128个字节中。

## 7.21 类成员函数(方法)

每次声明或创建类的新对象时，它都将生成数据成员的新实例。
但是每个成员函数只有一个实例。函数代码不用复制多份，因为相同的代码可以应用于类的所有实例。

下面两种方式速度一样快：
- 调用结构的成员函数
- 调用简单函数，以指针（或引用）为参数，该指针向此结构

例如:

```cpp
// Example 7.41
class S3 {
public:
   int a;
   int b;
   int Sum1() {return a + b;}
};
int Sum2(S3 * p) {return p->a + p->b;}
int Sum3(S3 & r) {return r.a + r.b;}
```
`Sum1`，`Sum2`和`Sum3`这三个函数完成相同的工作，效率也相同。
如果查看编译器生成的代码，您会注意到一些编译器将为这三个函数生成完全相同的代码。
`Sum1`有一个隐含的'`this`'指针，它与`Sum2`和`Sum3`中的`p`和`r`做同样的事情。
无论你是想让函数成为类的成员，还是给它一个指针或引用，指该向类或结构，都只是编程风格的问题。
有些编译器通过在寄存器而不是堆栈中传输'`this`'，使得`Sum1`在32位Windows中的效率略高于`Sum2`和`Sum3`。

## 7.22 Virtual member functions

Virtual functions are used for implementing polymorphic classes. Each instance of a
polymorphic class has a pointer to a table of pointers to the different versions of the virtual
functions. This so-called virtual table is used for finding the right version of the virtual
function at runtime. Polymorphism is one of the main reasons why object oriented programs
can be less efficient than non-object oriented programs. If you can avoid virtual functions
then you can obtain most of the advantages of object oriented programming without paying
the performance costs.

The time it takes to call a virtual member function is a few clock cycles more than it takes to
call a non-virtual member function, provided that the function call statement always calls the
same version of the virtual function. If the version changes then you may get a misprediction
penalty of 10 - 20 clock cycles. The rules for prediction and misprediction of virtual function
calls is the same as for switch statements, as explained on page 44.

The dispatching mechanism can be bypassed when the virtual function is called on an
object of known type, but you cannot always rely on the compiler bypassing the dispatch
mechanism even when it would be obvious to do so. See page 75.

Runtime polymorphism is needed only if it cannot be known at compile time which version
of a polymorphic member function is called. If virtual functions are used in a critical part of a
program then you may consider whether it is possible to obtain the desired functionality
without polymorphism or with compile-time polymorphism.

It is sometimes possible to obtain the desired polymorphism effect with templates instead of
virtual functions. The template parameter should be a class containing the functions that
have multiple versions. This method is faster because the template parameter is always
resolved at compile time rather than at runtime. Example 7.47 on page 59 shows an
example of how to do this. Unfortunately, the syntax is so kludgy that it may not be worth
the effort.


## 7.23 Runtime type identification (RTTI)

Runtime type identification adds extra information to all class objects and is not efficient. If
the compiler has an option for RTTI then turn it off and use alternative implementations.

## 7.24 Inheritance

An object of a derived class is implemented in the same way as an object of a simple class
containing the members of both parent and child class. Members of parent and child class
are accessed equally fast. In general, you can assume that there is hardly any performance
penalty to using inheritance.

There may be a slight degradation in code caching for the following reasons:

- The size of the parent class data members is added to the offset of the child class
members. The code that accesses data members with a total offset bigger than 127
bytes is slightly less compact. See page 54.

- The member functions of parent and child are typically stored in different modules.
This may cause a lot of jumping around and less efficient code caching. This
problem can be solved by making sure that functions which are called near each
other are also stored near each other. See page 90 for details.

Inheritance from multiple parent classes in the same generation can cause complications
with member pointers and virtual functions or when accessing an object of a derived class
through a pointer to one of the base classes. You may avoid multiple inheritance by making
objects inside the derived class:
```cpp
// Example 7.42a. Multiple inheritance
class B1; class B2;
class D : public B1, public B2 {
public:
   int c;
};
```
Replace with:
```cpp
// Example 7.42b. Alternative to multiple inheritance
class B1; class B2;
class D : public B1 {
public:
   B2 b2;
   int c;
};
```

## 7.25 Constructors and destructors

A constructor is implemented internally as a member function which returns a reference to
the object. The allocation of memory for a new object is not necessarily done by the
constructor itself. Constructors are therefore as efficient as any other member functions.
This applies to default constructors, copy constructors, and any other constructors.

A class doesn't need a constructor. A default constructor is not needed if the object doesn't
need initialization. A copy constructor is not needed if the object can be copied simply by
copying all data members. A simple constructor may be inlined for improved performance.

A copy constructor may be called whenever an object is copied by assignment, as a
function parameter, or as a function return value. The copy constructor can be a time
consumer if it involves allocation of memory or other resources. There are various ways to
avoid this wasteful copying of memory blocks, for example:

- Use a reference or pointer to the object instead of copying it
- Use a "move constructor" to transfer ownership of the memory block. This requires a
compiler with C++0x support.
- Make a member function or friend function or operator that transfers ownership of
the memory block from one object to another. The object that looses ownership of
the memory block should have its pointer set to NULL. There should of course be a
destructor that destroys any memory block that the object owns.

A destructor is as efficient as a member function. Do not make a destructor if it is not
necessary. A virtual destructor is as efficient as a virtual member function. See page 55.